In [2]:
import parselmouth
import librosa
import pyworld as pw
from pydub import AudioSegment
from sklearn.preprocessing import StandardScaler
import numpy as np
import os
import malaya_speech
from malaya_speech import Pipeline
from malaya_speech.utils.text import TextIDS
from glob import glob
import json

In [3]:
base_directory = '/home/husein/ssd2/audiobook'

In [4]:
import yaml

with open('config.yaml') as fopen:
    config = yaml.safe_load(fopen)
    
config

{'sampling_rate': 22050,
 'fft_size': 1024,
 'hop_size': 256,
 'win_length': None,
 'window': 'hann',
 'num_mels': 80,
 'fmin': 80,
 'fmax': 7600,
 'global_gain_scale': 1.0,
 'trim_silence': True,
 'trim_threshold_in_db': 60,
 'trim_frame_size': 2048,
 'trim_hop_size': 512}

In [5]:
import numpy as np

# https://github.com/TensorSpeech/TensorFlowTTS/blob/master/tensorflow_tts/utils/outliers.py
def is_outlier(x, p25, p75):
    """Check if value is an outlier."""
    lower = p25 - 1.5 * (p75 - p25)
    upper = p75 + 1.5 * (p75 - p25)
    return x <= lower or x >= upper


def remove_outlier(x, p_bottom: int = 25, p_top: int = 75):
    """Remove outlier from x."""
    p_bottom = np.percentile(x, p_bottom)
    p_top = np.percentile(x, p_top)

    indices_of_outliers = []
    for ind, value in enumerate(x):
        if is_outlier(value, p_bottom, p_top):
            indices_of_outliers.append(ind)

    x[indices_of_outliers] = 0.0
    x[indices_of_outliers] = np.max(x)
    return x

In [6]:
import re

_pad = 'pad'
_start = 'start'
_eos = 'eos'
_punctuation = "!'(),.:;? "
_special = '-'
_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

MALAYA_SPEECH_SYMBOLS = (
    [_pad, _start, _eos] + list(_special) + list(_punctuation) + list(_letters)
)

In [7]:
def tts_encode(string: str, add_eos: bool = True):
    r = [MALAYA_SPEECH_SYMBOLS.index(c) for c in string if c in MALAYA_SPEECH_SYMBOLS]
    if add_eos:
        r = r + [MALAYA_SPEECH_SYMBOLS.index('eos')]
    return r

In [8]:
from unidecode import unidecode
import malaya

normalizer = malaya.normalize.normalizer()
sentence_tokenizer = malaya.text.function.split_into_sentences

tokenizer = TextIDS(
    pad_to=None,
    understand_punct=False,
    is_lower=True,
    normalizer=normalizer,
    sentence_tokenizer=sentence_tokenizer,
)

In [9]:
tokenizer.normalize('saya tak suka ayam.')

('saya tak suka ayam .',
 array([58, 40, 64, 40, 13, 59, 40, 50, 13, 58, 60, 50, 40, 13, 40, 64, 40,
        52, 13,  9]))

In [10]:
from unidecode import unidecode

def put_spacing_num(string):
    string = re.sub('[A-Za-z]+', lambda ele: ' ' + ele[0] + ' ', string)
    return re.sub(r'[ ]+', ' ', string).strip()

def convert_to_ascii(string):
    return unidecode(string)

def collapse_whitespace(string):
    return re.sub(_whitespace_re, ' ', string)

def cleaning(string, normalize = True, add_eos = False):
    sequence = []
    string = convert_to_ascii(string)
    string = string.replace('&', ' dan ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = string
    string = put_spacing_num(string)
    string = ''.join([c for c in string if c in MALAYA_SPEECH_SYMBOLS])
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

In [11]:
with open(os.path.join(base_directory, 'comma-true-case.json')) as fopen:
    data = json.load(fopen)
    
audiobook = []
directory = 'noisy-audiobook'
os.makedirs(directory, exist_ok=True)
for k, v in data.items():
    try:
        wav = os.path.join(base_directory, k)
        if os.path.exists(wav) and len(cleaning(v['true_case'])) > 2:
            audiobook.append((wav, cleaning(v['true_case']), directory))
    except Exception as e:
        print(e, k, v)
        
len(audiobook)

2968

In [12]:
import malaya_speech
import soundfile as sf
from malaya_speech import Pipeline
from datasets import Audio
from tqdm import tqdm

def process(txts, 
            start_silent_trail = int(0.15 * config['sampling_rate']),
            middle_silent_trail = int(0.2 * config['sampling_rate']),
            end_silent_trail = int(0.2 * config['sampling_rate']),
            process_middle_silent = True,
            maxlen = 20):
    
    reader = Audio(sampling_rate = 22050)
    vad = malaya_speech.vad.webrtc()
    txts = txts[0]
    audios, text_ids = [], []

    for f in tqdm(txts):
        
        directory = f[2]
        text = f[1]
        f = f[0]
        
        if '.mp3' in f:
            sound = AudioSegment.from_mp3(f)
            samples = sound.get_array_of_samples()
            samples = np.array(samples)
            samples = malaya_speech.astype.int_to_float(samples)
            audio = malaya_speech.resample(samples, sound.frame_rate, 22050)
            
        else:
            audio = reader.decode_example(reader.encode_example(f))['array']

        if config['trim_silence']:
            y_= malaya_speech.resample(audio, config['sampling_rate'], 16000)
            y_ = malaya_speech.astype.float_to_int(y_)
            frames = list(malaya_speech.generator.frames(audio, 30, config['sampling_rate']))
            frames_ = list(malaya_speech.generator.frames(y_, 30, 16000, append_ending_trail = False))
            frames_webrtc = [(frames[no], vad(frame)) for no, frame in enumerate(frames_)]
            grouped_deep = malaya_speech.group.group_frames(frames_webrtc)
            grouped_deep = malaya_speech.group.group_frames_threshold(grouped_deep, 0.15)
            r = []
            for no, g in enumerate(grouped_deep):
                if g[1]:
                    g = g[0].array
                else:
                    if no == 0:
                        g = g[0].array[-start_silent_trail:]
                    elif no == (len(grouped_deep) - 1):
                        g = g[0].array[:end_silent_trail]
                    else:
                        if process_middle_silent:
                            g = np.concatenate([g[0].array[:middle_silent_trail], g[0].array[-middle_silent_trail:]])
                        else:
                            g = g[0].array
                        
                r.append(g)
            audio = np.concatenate(r)
        
        if (len(audio) / config['sampling_rate']) > maxlen:
            continue
        
        if (len(audio) / config['sampling_rate']) < 0.5:
            continue
            
        audio = np.pad(audio, (0, config["fft_size"]), mode="edge")
        new_f = '-'.join(f.split('/')[-2:]).replace('.mp3', '.wav')
        new_f = os.path.join(directory, new_f)
#         new_f = f'{directory}/{f}'.replace('.wav', '.mp3')
#         torchaudio.save(new_f, torch.tensor(audio.astype('float32')).unsqueeze(0), 
#                         22050, format='mp3')
        
        sf.write(new_f, audio, 22050)
        audios.append(new_f)
        text_ids.append(text)
    
    return [[audios, text_ids]]

In [13]:
!rm -rf noisy-audiobook
!mkdir noisy-audiobook

In [17]:
import mp

audios, text_ids = [], []
batch_size = 5000
results = mp.multiprocessing(audiobook, process, cores = 5, returned = True)    
for result in results:
    audios.extend(result[0])
    text_ids.extend(result[1])

100%|█████████████████████████████████████████| 593/593 [02:19<00:00,  4.25it/s]


In [18]:
# '(Audiobook) Mengenali Imam Mahdi'
# 'HARRY POTTER DAN BILIK RAHSIA'
# 'BUKAN KERANA AKU TAK CINTA'
# 'SIRI BEDAH'

speakers_id = []
for f in audios:
    if '(Audiobook) Mengenali Imam Mahdi' in f:
        i = 0
    if 'HARRY POTTER DAN BILIK RAHSIA' in f or 'BUKAN KERANA AKU TAK CINTA' in f:
        i = 1
    if 'SIRI BEDAH' in f:
        i = 2
    speakers_id.append(i)
    
len(speakers_id) == len(audios)

True

In [20]:
from tqdm import tqdm
import random

data = []
for i in tqdm(range(len(audios))):
    data.append((os.path.join(os.getcwd(), audios[i]), speakers_id[i], text_ids[i]))
    
random.shuffle(data)

100%|███████████████████████████████████| 2951/2951 [00:00<00:00, 322353.08it/s]


In [21]:
len(data)

2951

In [23]:
import json

with open('multispeaker-clean-v2-vits.json', 'w') as fopen:
    json.dump(data, fopen)

In [1]:
import json

with open('multispeaker-clean-v2-vits.json') as fopen:
    data = json.load(fopen)

In [2]:
data

[['/home/husein/speech-bahasa/noisy-audiobook/BUKAN KERANA AKU TAK CINTA (BAB 4) - AUDIOBOOK [u9ulw_KXsdA]_-80.wav',
  1,
  'Terterpaku seketika , jebat pandang Mastura , namun seperti tadi , Mastura masih tetap menulis sesuatu pada dirinya , seolah olah ,'],
 ['/home/husein/speech-bahasa/noisy-audiobook/HARRY POTTER DAN BILIK RAHSIA (BAB 3) - MALAY AUDIOBOOK [ikQilu3cYsY]_-169.wav',
  1,
  'Kalau kamu fikir kamu lebih pandai daripada lokat , pergilah teruskan kerja kamu , tapi awal , kalau masih ada nombor lagi di taman masa mak periksaan ,'],
 ['/home/husein/speech-bahasa/noisy-audiobook/🛑 ｜ SIRI BEDAH BUKU SAHAM - THE SECRETS OF TRADING THE FIRST PULLBACK (ELWIN NG) ｜ Dr. Al-Aarifin [l8-Pf5iwhdk]_-149.wav',
  2,
  'Kawasan kita pada hari ini , InsyaAllah ,'],
 ['/home/husein/speech-bahasa/noisy-audiobook/HARRY POTTER DAN BILIK RAHSIA (BAB 5) - MALAY AUDIOBOOK [VNaA5Ghb4FU]_-42.wav',
  1,
  'Satu saat , sudah berlepas ,'],
 ['/home/husein/speech-bahasa/noisy-audiobook/🛑 ｜ SIRI BEDAH 

In [7]:
with open('multispeaker-clean-vits.json') as fopen:
    v1 = json.load(fopen)

In [8]:
combined = data[:]
for d in v1:
    combined.append([d[0], d[1] + 3, d[2]])

In [13]:
len(set([d[1] for d in combined]))

12

In [12]:
with open('multispeaker-clean-v3-vits.json', 'w') as fopen:
    json.dump(combined, fopen)

In [2]:
import json

with open('multispeaker-clean-v3-vits.json') as fopen:
    data = json.load(fopen)

In [3]:
!pwd

/home/ubuntu/server2/speech-bahasa


In [4]:
data[0]

['/home/husein/speech-bahasa/noisy-audiobook/BUKAN KERANA AKU TAK CINTA (BAB 4) - AUDIOBOOK [u9ulw_KXsdA]_-80.wav',
 1,
 'Terterpaku seketika , jebat pandang Mastura , namun seperti tadi , Mastura masih tetap menulis sesuatu pada dirinya , seolah olah ,']

In [5]:
for i in range(len(data)):
    data[i][0] = data[i][0].replace('/home/husein/speech-bahasa', '/home/ubuntu/server2/speech-bahasa')

In [7]:
with open('multispeaker-clean-v3-vits-nfs.json', 'w') as fopen:
    json.dump(data, fopen)